# Tribble Dashboard

## Worktime Analyze
Enter the timespan to be analyzed:

In [ ]:
timespan_begin="30.03.2023"
timespan_end  ="03.04.2023"

Execute this cell get hours sorted by day and symbol:

In [ ]:
from datetime import datetime, timedelta
import pytz
import sqlite3
import os
import time
from IPython.display import clear_output

def analyse_day(timeslot_list):
    timeslot_dict = {}
    for timeslot in timeslot_list:
        begin_tz = datetime.strptime(timeslot[0], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Berlin'))
        end_tz = datetime.strptime(timeslot[1], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Berlin'))
        difference = end_tz - begin_tz
        
        if timeslot[2] not in timeslot_dict.keys():
            timeslot_dict[timeslot[2]] = timedelta(days=0)
        timeslot_dict[timeslot[2]] = timeslot_dict[timeslot[2]] + difference
    return timeslot_dict   

tribble_db_fqn = f"{os.getenv('JUBILANT_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

timespan_begin_obj = datetime.strptime(timespan_begin, "%d.%m.%Y")
timespan_end_obj = datetime.strptime(timespan_end, "%d.%m.%Y")
timespan_end_obj_loop = timespan_end_obj + timedelta(days=1)

current_day = timespan_begin_obj

while current_day != timespan_end_obj_loop:
    current_day_tz = current_day.astimezone(pytz.timezone('Europe/Berlin'))
    current_day_utc_begin = current_day_tz.astimezone(pytz.utc)
    current_day_utc_end = current_day_utc_begin + timedelta(days=1)
    
    current_day_utc_begin_string = current_day_utc_begin.strftime("%Y-%m-%d %H:%M:%S")
    current_day_utc_end_string = current_day_utc_end.strftime("%Y-%m-%d %H:%M:%S")
    
    res = cur.execute(f"SELECT timeslot_begin, timeslot_end, symbol FROM worktime w WHERE DATETIME(timeslot_begin) >= DATETIME('{current_day_utc_begin_string}') AND DATETIME(timeslot_begin) < DATETIME('{current_day_utc_end_string}');")
    results_list = res.fetchall()
    
    analyse_result = analyse_day(results_list)
    
    if analyse_result != {}:
        print(f'\n---------------\ndate:{current_day.strftime("%d.%m.%Y")}\n---------------')
        for key, value in analyse_result.items():
            print(key, value)
    
    current_day = current_day + timedelta(days=1)
con.close()


In [ ]:
from datetime import datetime, timedelta
import pytz
import sqlite3
import os
import time
import plotly.express as px
import pandas as pd

def analyse_day(timeslot_list):
    timeslot_dict_list = []
    Task_id_iterator = 0
    for timeslot in timeslot_list:
        
        begin_tz = datetime.strptime(timeslot[0], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Berlin'))
        end_tz = datetime.strptime(timeslot[1], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Europe/Berlin'))
        
        timeslot_dict_list.append(dict(Task=f"Job_{Task_id_iterator}", Start=begin_tz, Finish=end_tz, Resource=timeslot[2]))
        
        Task_id_iterator = Task_id_iterator + 1
    
    df = pd.DataFrame(timeslot_dict_list)
    return df   

tribble_db_fqn = f"{os.getenv('JUBILANT_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

timespan_begin_obj = datetime.strptime(timespan_begin, "%d.%m.%Y")
timespan_end_obj = datetime.strptime(timespan_end, "%d.%m.%Y")
timespan_end_obj_loop = timespan_end_obj + timedelta(days=1)

current_day = timespan_begin_obj

while current_day != timespan_end_obj_loop:
    current_day_tz = current_day.astimezone(pytz.timezone('Europe/Berlin'))
    current_day_utc_begin = current_day_tz.astimezone(pytz.utc)
    current_day_utc_end = current_day_utc_begin + timedelta(days=1)
    
    current_day_utc_begin_string = current_day_utc_begin.strftime("%Y-%m-%d %H:%M:%S")
    current_day_utc_end_string = current_day_utc_end.strftime("%Y-%m-%d %H:%M:%S")
    
    res = cur.execute(f"SELECT timeslot_begin, timeslot_end, symbol FROM worktime w WHERE DATETIME(timeslot_begin) >= DATETIME('{current_day_utc_begin_string}') AND DATETIME(timeslot_begin) < DATETIME('{current_day_utc_end_string}');")
    results_list = res.fetchall()
    
    if results_list != []:

        df = analyse_day(results_list)

        # https://plotly.com/python/gantt/
        fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
        print(f'\n---------------\ndate:{current_day.strftime("%d.%m.%Y")}\n---------------')
        fig.show()

    current_day = current_day + timedelta(days=1)
con.close()